# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Ristorante Il Fogolar Là di Moret - Friuli Venezia Giulia

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187814-d2144883-Reviews-Ristorante_Il_Fogolar_di_Moret-Udine_Province_of_Udine_Friuli_Venezia_Giulia.html

Home --> Ristoranti --> Firenze --> Best_ranking--> SENZ_ALTRO_Bistrot

In [0]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [0]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187814-d2144883-Reviews-Ristorante_Il_Fogolar_di_Moret-Udine_Province_of_Udine_Friuli_Venezia_Giulia.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [0]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [0]:
URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187814-d2144883-Reviews-or10-Ristorante_Il_Fogolar_di_Moret-Udine_Province_of_Udine_Friuli_Venezia_Giuli.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187814-d2144883-Reviews-or'
URL_p2 = '0-Ristorante_Il_Fogolar_di_Moret-Udine_Province_of_Udine_Friuli_Venezia_Giuli.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in range(1,total_pages):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
total_pages:  52
Nomi:  516
Commenti:  516
Voti:  516
Scritto da:  516


In [0]:
# display(all_reviews)

In [0]:
# print(all_names)

In [0]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [0]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [0]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 516 
 Recensioni analizzabili: 231 
 Recensioni scartate: 285 



['RECENSIONE_TROPPO_LUNGA',
 "Lo consiglio a chi desidera mangiare in un locale riservato ed esclusivo con ampio parcheggio e all'occorrenza la SPA",
 'RECENSIONE_TROPPO_LUNGA',
 'Ottimo ristorante della tradizione friulana con un occhio all innovazione, Renato l anima del ristorante...consigliatissimo senza ombra di dubbio',
 'RECENSIONE_TROPPO_LUNGA',
 'Ottima accoglienza, ambiente curato nei dettagli, piatti ben strutturati con attento impiego delle materie prime, realizzazioni e presentazioni di livello, tempi perfetti ... rapporto q/p ottimo ... che dire di piû??',
 'Ottimo ristorante, splendida accoglienza, personale qualificato e di grande cortesia, ambiente caldo e curato nei dettagli. Cibo Squisito con particolare attenzione agli ingredienti: sani, freschi e deliziosi. Tornerò sicuramente!',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA']

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [0]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['antonelli79', '645ivano', 'Silvio T', 'luca c', 'Giovik66', 'AndreaF463', 'andrejpeiboschi', 'GianniOvan1', 'paolo c', 'ernesto p', 'Elena E', 'Monik26588', 'maurocJ1313DA', 'ilDemonio21', 'FabioFinotti', '391ariannaa', 'lauramarco30', 'Amelia P', 'Silvano A', 'ElyMark1', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'J6357KKmatteos', 'Giovanni C', 'dado7419', 'Giulymoon04', 'DaPiazzola', 'ChuckC717', 'andrea k k', 'emilio d', 'CavialeMusone', '91patric', 'AgneseC', 'T6127AJandreat', 'marco2974', 'js245345', 'mirkob128', 'macecco', 'Kaizen54', 'umbertoc427', '_cosutina', 'leonardo m', 'Elbuitre78', 'pravisap', 'sergio m', 'Max_V_Ud', 'Valter P', 'amicobulldog', 'Annalisa I', 'Ebiz22', 'GIBI1158', '805filippom', 'Vanderwalls', 'PatrizioD959', 'francescoarcana', 'MirkoMK90', 'Copet86', 'lucabA6898YY', 'Paola T', 'gianluis51', 'Sara M', 'G2452AHnicolap', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [0]:
!pip install gender_guesser

In [0]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unk

In [0]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [0]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  516
Commenti:  516
Voti:  516
Mobile:  516


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [0]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  38


[['Amelia',
  'F',
  "Cibo di ottima qualità. Ambiente pulito anche se un po' asettico. Serrvizio buono, attesa un po' lunga.",
  '4',
  'PC'],
 ['Silvano',
  'M',
  'Il ristorante che preferisco in assoluto con piatti ottimi e presentati in modo originale. E per chiudere il gelato alla crema mantecato. Da non perdere.',
  '5',
  'PC'],
 ['Valter',
  'M',
  'Struttura appena alla periferia di Udine, ambiente accogliente e ben curato. Menù di carne e pesce. Abbiamo scelto un antipastino di misto mare, molto buono, poi un risotto di mare. Tutto molto sfizioso, risotto splendido, vini regionali super, conto adeguato.',
  '5',
  'PC'],
 ['Annalisa',
  'F',
  'Ambiente caldo e accogliente, al tempo stesso raffinato e romantico.\nPersonale oltremodo gentile, ti fa sentire a casa.\nCibo superlativo, particolare e ricercato nei gusti e negli abbinamenti. Enorme disponibilità di vini.\nUna garanzia.',
  '5',
  'Mobile'],
 ['Sara',
  'F',
  'Un luogo che trasmette serenità e garantisce qualità -

### Creiamo la tabella

In [0]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [0]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_2.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [0]:
partial_dataframe = pd.read_excel("RISTORANTE_2.xlsx")

display(partial_dataframe)

,Genere,Nomi,Recensioni,Scritta da,Voti
0,F,Amelia,Cibo di ottima qualità. Ambiente pulito anche ...,PC,4
1,M,Silvano,Il ristorante che preferisco in assoluto con p...,PC,5
2,M,Valter,"Struttura appena alla periferia di Udine, ambi...",PC,5
3,F,Annalisa,"Ambiente caldo e accogliente, al tempo stesso ...",Mobile,5
4,F,Sara,Un luogo che trasmette serenità e garantisce q...,Mobile,5
5,M,Massimo,Ogni piatto assaggiato è stato spettacolare. I...,PC,5
6,F,Renata,"Cena con amici, qualita' del cibo ottimo, scel...",PC,5
7,F,Maria,Ottimo!!! ambiente caldo e accogliente cibo ot...,Mobile,5
8,F,Elena,"Fantasia, gusto e gentilezza sono le prime tre...",Mobile,5
9,F,Arianna,Sono stata sabato in occasione di un pranzo di...,Mobile,5
